Import modules and define constants.

In [1]:
import matplotlib.pyplot as plt
import numpy as np
import json
import os
import modules.analyzer as an

# Constants
SETTING_PATH = "draw-figures-settings.json"
DATASET_PATHS = [
    "outputs-figures",
    "outputs-reproduce/cov-wt-minimizers",
]

Import the setting file (which is shared with `draw-figures.py` script).

In [2]:
# Read the settings
assert os.path.exists(SETTING_PATH), "Setting file does not exist!"
with open(SETTING_PATH, "r") as f:
    settings = json.load(f)

Import the minimizer files. These files must be prepared using `main.py` program. See README for the details.

In [3]:
DB_MINIMIZER: dict[str, dict] = {}
for path in DATASET_PATHS:
    for root, dirs, files in os.walk(path):
        for file in files:
            if not file.endswith(".csv"):
                continue
            cur_path = os.path.join(root, file)

            # Open the metadata file
            cur_path_meta = cur_path[:-4] + "-metadata.json"
            if not os.path.exists(cur_path_meta):
                print(f"Metadata file not found for {cur_path}. Skipping...")
                continue
            # Open the metadata and validate the file schema
            with open(cur_path_meta, "r") as f:
                metadata = json.load(f)
            for value in ["escape", "result", "virus", "strain", "times"]:
                if value not in metadata:
                    continue
            # Create a unique serial to prevent duplicates
            serial = f"({metadata['escape']})-({metadata['result']})-({metadata['virus']})-({metadata['strain']})"
            if serial in DB_MINIMIZER:
                print(f"Duplicate entry found. Skipping...")
                continue

            # Open the CSV file and load the data
            with open(cur_path, "r") as f:
                data = np.loadtxt(f, delimiter=",", skiprows=1, dtype=float)
            # Add to the dictionary
            DB_MINIMIZER[serial] = metadata | {"data": data}
            print(f"Loaded {cur_path} with metadata.")

Loaded outputs-reproduce/cov-wt-minimizers/hie/wt2dms-min.csv with metadata.
Loaded outputs-reproduce/cov-wt-minimizers/hie/wt2delta-min.csv with metadata.
Loaded outputs-reproduce/cov-wt-minimizers/hie/wt2combined-min.csv with metadata.
Loaded outputs-reproduce/cov-wt-minimizers/hie/wt2beta-min.csv with metadata.
Loaded outputs-reproduce/cov-wt-minimizers/hie/wt2omicron-min.csv with metadata.
Loaded outputs-reproduce/cov-wt-minimizers/hie/wt2alpha-min.csv with metadata.
Loaded outputs-reproduce/cov-wt-minimizers/esm2-3b-cos/wt2beta-min.csv with metadata.
Loaded outputs-reproduce/cov-wt-minimizers/esm2-3b-cos/wt2alpha-min.csv with metadata.
Loaded outputs-reproduce/cov-wt-minimizers/esm2-3b-cos/wt2omicron-min.csv with metadata.
Loaded outputs-reproduce/cov-wt-minimizers/esm2-3b-cos/wt2dms-min.csv with metadata.
Loaded outputs-reproduce/cov-wt-minimizers/esm2-3b-cos/wt2delta-min.csv with metadata.
Loaded outputs-reproduce/cov-wt-minimizers/esm2-3b-cos/wt2combined-min.csv with metadata.


Export the list plots for the optimal coefficient ($\gamma$) of CLIMB regularizer.

In [4]:
values = list(entry["value"] for entry in settings["datasets"])
labels = list(entry["label"] for entry in settings["datasets"])
v2l_map = {entry["value"]: entry["label"] for entry in settings["datasets"]}  # Value to label map

for escape in settings["validation-sets"]:
    # Initialize the plot
    plt.figure(figsize=(10, 6))

    db_filtered = {}
    for entry in DB_MINIMIZER.values():
        if entry["escape"] != escape:
            continue
        value = entry["result"]
        if value not in values:
            continue
        strain = entry["strain"]
        times = entry["data"][:, 0]
        db_filtered[value] = entry

    values_filtered = [value for value in values if value in db_filtered]
    labels_filtered = [v2l_map[value] for value in values_filtered]

    for label, value in zip(labels_filtered, values_filtered):
        entry = db_filtered[value]
        x_p = entry["data"][:, 3]  # Optimal coefficient for CLIMB (γ)
        plt.plot(times, x_p, linestyle="-", linewidth=3, alpha=0.85, label=label)

    # Axes and labels
    plt.ylim(0, 1.01)
    plt.xlabel("Evolutionary Parameter (T)")
    plt.ylabel("Optimal Coefficient for CLIMB (γ)")
    plt.title(f"Optimal CLIMB coefficient (γ) over T ({escape})")
    plt.legend()
    plt.tight_layout()

    # Prepare output directory
    virus_prefix = an.STRAIN_DATA[strain]["prefix"]
    escape_prefix = an.ESCAPE_DATA[escape]["prefix"]
    output_dir = os.path.join("outputs-figures", virus_prefix)
    if not os.path.exists(output_dir):
        os.makedirs(output_dir, exist_ok=True)

    filename = f"optcoef-(e={escape_prefix}).pdf"
    output_path = os.path.join(output_dir, filename)
    plt.savefig(output_path, dpi=300)
    plt.close()
    print(f"Saved figure to {output_path}")

Saved figure to outputs-figures/cov-wt/optcoef-(e=wt2dms).pdf
Saved figure to outputs-figures/cov-wt/optcoef-(e=wt2alpha).pdf
Saved figure to outputs-figures/cov-wt/optcoef-(e=wt2beta).pdf
Saved figure to outputs-figures/cov-wt/optcoef-(e=wt2delta).pdf
Saved figure to outputs-figures/cov-wt/optcoef-(e=wt2omicron).pdf
Saved figure to outputs-figures/cov-wt/optcoef-(e=wt2combined).pdf
